In [13]:
from sage.all import *
from pwn import *
import ast

HOST,PORT = "challenge.imxbt.cn" , 31501
io = remote(HOST, PORT)

# 接收 A (格式: [[...]])
io.recvuntil(b"[[")
raw_A_str = "[[" + io.recvuntil(b"]]").decode()
A_list = ast.literal_eval(raw_A_str)
# 接收 b (格式: [...])
io.recvuntil(b"[")
raw_b_str = "[" + io.recvuntil(b"]").decode()
b_list = ast.literal_eval(raw_b_str)

def solve_sis():

    # 定义矩阵和向量
    # 这里使用 ZZ (整数环)
    A = Matrix(ZZ, A_list)
    b = vector(ZZ, b_list)
    
    nrows = A.nrows()   # 16
    ncols = A.ncols()   # 32
    
    print(f"[*] 正在初始化格矩阵 (Rows: {nrows}, Cols: {ncols})...")

    # -----------------------------------------------------
    # 构造格 (Lattice Construction) - Embedding Method
    # -----------------------------------------------------
    # 我们构造一个矩阵 M，形式如下：
    # [ I  |  K * A^T ]
    # [ 0  | -K * b^T ]
    #
    # 这里的 K 是一个巨大的权重。
    # 如果我们有一个向量 v = (x_1, ..., x_32, 1)
    # v * M = (x, K(Ax - b))
    # 如果 Ax = b，那么结果向量的右半部分将全部为 0。
    # LLL 算法会寻找短向量，因为 x 很小而 K 很大，算法会优先强制右半部分为 0。
    # -----------------------------------------------------

    # 权重 K：必须足够大，使得满足方程约束比减小 x 的数值更重要。
    # x 的最大值约 10^5，K 取 10^10 以上比较安全。
    K = 2**60 
    
    # 构造矩阵 M
    # 维度：(32个变量 + 1个常数项) 行， (32列存x + 16列存约束) 列
    M = Matrix(ZZ, ncols + 1, ncols + nrows)
    
    # 1. 左上角：单位矩阵 I (用于保持 x 的值)
    M.set_block(0, 0, Matrix.identity(ncols))
    
    # 2. 右上角：K * A^T (方程系数)
    M.set_block(0, ncols, K * A.transpose())
    
    # 3. 最后一行：常数项处理
    # 左下角是 0 (默认就是0，不用设)
    # 右下角是 -K * b (方程的常数项移项)
    for i in range(nrows):
        M[ncols, ncols + i] = -K * b[i]

    print("[*] 正在运行 LLL 算法 (这可能需要几秒钟)...")
    
    # 运行 LLL 规约
    L_reduced = M.LLL()
    
    print("[*] LLL 完成，正在搜索解向量...")

    solution = None
    
    # 遍历规约后基底的每一行
    for row in L_reduced:
        # 提取向量的右半部分 (应该全为 0)
        check_part = row[ncols:]
        
        # 提取向量的左半部分 (可能是解 x)
        x_part = row[:ncols]
        
        # 检查1：约束部分是否全为 0
        if not check_part.is_zero():
            continue
            
        # 检查2：x 部分不能全为 0
        if x_part.is_zero():
            continue
            
        # 修正符号：LLL 可能会返回 -x
        # 题目中 x 是正整数，所以如果我们发现大部分是负数，就取反
        # 简单的启发式：看第一个非零元素是否为负
        if x_part[0] < 0:
            x_part = -x_part
            
        # 检查3：代回验证 Ax = b
        # 将 vector 转换为 list 方便计算
        x_candidate = list(x_part)
        
        # 验证
        recalc_b = [sum(A_list[i][j] * x_candidate[j] for j in range(ncols)) for i in range(nrows)]
        
        if recalc_b == b_list:
            print("[+] 成功找到解！")
            solution = x_candidate
            break
    
    if solution:
        x = " ".join(map(str, solution))
        print(x)
        io.sendlineafter(b"Enter your solution: ", x.encode())
        print(io.recvall().decode())
        io.close()
    else:
        print("[-] 未找到解。可能原因：")
        print("2. 权重 K 不够大 (尝试增大 K)")

# 执行
solve_sis()

[x] Opening connection to challenge.imxbt.cn on port 31501
[x] Opening connection to challenge.imxbt.cn on port 31501: Trying 119.188.240.24
[+] Opening connection to challenge.imxbt.cn on port 31501: Done
[*] 正在初始化格矩阵 (Rows: 16, Cols: 32)...
[*] 正在运行 LLL 算法 (这可能需要几秒钟)...
[*] LLL 完成，正在搜索解向量...
[+] 成功找到解！
35 19074 81893 58041 76514 61485 8973 5519 10007 11369 2339 31799 2029 68360 15536 10160 25423 12915 77596 45464 113066 76983 40770 92246 37144 90188 24336 70938 28046 87776 65380 31189
[x] Receiving all data
[x] Receiving all data: 0B
[x] Receiving all data: 71B
[+] Receiving all data: Done (71B)
[*] Closed connection to challenge.imxbt.cn port 31501
Bravo! Here is your flag:
LILCTF{a23bf0c6-b7d7-40d9-8cd0-54769555829f}

